<a href="https://colab.research.google.com/github/zaid404/conv-docx-xlsx-to-pdf/blob/main/img2tex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pytesseract pillow opencv-python flask flask-ngrok


In [ ]:
from flask import Flask, render_template, request, redirect, url_for
from pyngrok import ngrok
import pytesseract
from PIL import Image
import cv2
import numpy as np
import os

app = Flask(__name__)

# Add your ngrok authtoken (replace 'YOUR_AUTHTOKEN' with your actual authtoken)
!ngrok config add-authtoken YOUR_AUTHTOKEN

# Start ngrok tunnel on port 5000
public_url = ngrok.connect(5000)
print(f" * ngrok tunnel available at: {public_url}")

def preprocess_image(image):
    """
    Preprocess the image for better OCR accuracy.
    """
    # Convert to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply a binary threshold to the image
    _, thresh_image = cv2.threshold(gray_image, 150, 255, cv2.THRESH_BINARY)

    return thresh_image

def ocr_image(image_path):
    """
    Perform OCR on the given image.
    """
    # Read the image using OpenCV
    image = cv2.imread(image_path)

    # Preprocess the image
    processed_image = preprocess_image(image)

    # Convert the processed image to PIL format
    pil_image = Image.fromarray(processed_image)

    # Perform OCR using pytesseract
    extracted_text = pytesseract.image_to_string(pil_image)

    return extracted_text

@app.route('/', methods=['GET', 'POST'])
def upload_image():
    if request.method == 'POST':
        # Check if a file is uploaded
        if 'file' not in request.files:
            return redirect(request.url)
        file = request.files['file']
        if file.filename == '':
            return redirect(request.url)
        if file:
            # Save the file to a temporary location
            image_path = os.path.join('/content/', file.filename)
            file.save(image_path)

            # Perform OCR on the uploaded image
            text = ocr_image(image_path)

            return render_template('result.html', extracted_text=text)
    return render_template('upload.html')

@app.route('/result')
def result():
    return render_template('result.html')

# Run the Flask app
if __name__ == "__main__":
    app.run()


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
 * ngrok tunnel available at: NgrokTunnel: "https://bcd9-34-90-46-10.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
ERROR:__main__:Exception on / [GET]
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 2529, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 1825, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 1823, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 1799, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
  File "<ipython-input-10-6d29e7ecec69>", line 66, in upload_image
    return render_template('upload.html')
  File "/usr/local/lib/python3.10/dist-packag

In [ ]:
from flask import Flask, render_template_string, request, redirect, url_for
from pyngrok import ngrok
import pytesseract
from PIL import Image
import cv2
import numpy as np
import os

app = Flask(__name__)

# Add your ngrok authtoken (replace 'YOUR_AUTHTOKEN' with your actual authtoken)
!ngrok config add-authtoken YOUR_AUTHTOKEN

# Start ngrok tunnel on port 5000
public_url = ngrok.connect(5000)
print(f" * ngrok tunnel available at: {public_url}")

def preprocess_image(image):
    """
    Preprocess the image for better OCR accuracy.
    """
    # Convert to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply a binary threshold to the image
    _, thresh_image = cv2.threshold(gray_image, 150, 255, cv2.THRESH_BINARY)

    return thresh_image

def ocr_image(image_path):
    """
    Perform OCR on the given image.
    """
    # Read the image using OpenCV
    image = cv2.imread(image_path)

    # Preprocess the image
    processed_image = preprocess_image(image)

    # Convert the processed image to PIL format
    pil_image = Image.fromarray(processed_image)

    # Perform OCR using pytesseract
    extracted_text = pytesseract.image_to_string(pil_image)

    return extracted_text

# HTML template for the upload page
upload_page = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Image OCR</title>
</head>
<body>
    <h1>Upload an Image for OCR</h1>
    <form method="POST" enctype="multipart/form-data">
        <input type="file" name="file" accept="image/*">
        <button type="submit">Upload</button>
    </form>
</body>
</html>
"""

# HTML template for the result page
result_page = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>OCR Result</title>
</head>
<body>
    <h1>Extracted Text</h1>
    <p>{{ extracted_text }}</p>
    <a href="{{ url_for('upload_image') }}">Go Back</a>
</body>
</html>
"""

@app.route('/', methods=['GET', 'POST'])
def upload_image():
    if request.method == 'POST':
        # Check if a file is uploaded
        if 'file' not in request.files:
            return redirect(request.url)
        file = request.files['file']
        if file.filename == '':
            return redirect(request.url)
        if file:
            # Save the file to a temporary location
            image_path = os.path.join('/content/', file.filename)
            file.save(image_path)

            # Perform OCR on the uploaded image
            text = ocr_image(image_path)

            return render_template_string(result_page, extracted_text=text)
    return render_template_string(upload_page)

@app.route('/result')
def result():
    return render_template_string(result_page)

# Run the Flask app
if __name__ == "__main__":
    app.run()


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
 * ngrok tunnel available at: NgrokTunnel: "https://f8b0-34-90-46-10.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [31/Aug/2024 02:12:03] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [31/Aug/2024 02:12:04] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [31/Aug/2024 02:12:11] "POST / HTTP/1.1" 200 -


In [ ]:
!apt-get update
!apt-get install -y tesseract-ocr


Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:6 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:7 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Get:8 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Ign:10 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,425 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2,498 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]

In [ ]:
!pip install pyngrok

from pyngrok import ngrok

# Add your ngrok authtoken
!ngrok config add-authtoken authtoken

# Start ngrok tunnel on port 5000
public_url = ngrok.connect(5000)  # Port as an integer, not a string
print(f" * ngrok tunnel available at: {public_url}")

# Now, run your Flask app
from flask import Flask

app = Flask(__name__)

@app.route("/")
def hello():
    return "Hello, world!"

if __name__ == "__main__":
    app.run()


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
 * ngrok tunnel available at: NgrokTunnel: "https://40f9-34-90-46-10.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [31/Aug/2024 02:02:20] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [31/Aug/2024 02:02:21] "GET /favicon.ico HTTP/1.1" 404 -
